In [ ]:
from scripts.functions import *
from parameters import *

to use your own api keys and parameters, copy paste the `parameters.py.dist` file in the same folder and remove the `.dist` extention. You can then replace the string with your own keys. only the .dist will be pushed to the dist git rep. 

# 1. Search items


### Create bounding box from centroids

In [ ]:
# Specify a projected EPSG for the centroids file
# EPSG:21148 is for Indonesia.
samples_gdf = read_from_centroids(projected_epsg='YOUR_EPSG', buffer=350, sep=';')

### Connect to client

In [ ]:
client = api.ClientV1(api_key=PLANET_API_KEY)

### Define filters

In [ ]:
# define test data for the filter
start_date = datetime.datetime(2017, 1, 1)
stop_date = datetime.datetime(2020, 12, 31)
cloud_cover_lte = 0.10
minimum_covered_area = 90 # included

### Define scores

In [ ]:
# item_type_score
item_type_score = {
    'PSScene4Band':10, 
    'PSScene3Band':0, 
    'PSOrthoTile':8,
    'REOrthoTile':0,
    'SkySatScene':0,
}

# season score
months_score = {
    1: 0, 7:8,
    2: 0, 8:10,
    3: 0, 9:10,
    4: 7, 10:8,
    5: 7, 11:0,
    6: 7, 12:0,
}

# cloud_score

def cloud_score(cloud_cover):
    """ Define the cloud cover threshold and score
    
    1 = 1%
    
    """
    cloud_cover = cloud_cover*100
    
    if cloud_cover == 0:
        return 10
    elif cloud_cover <= 5 and cloud_cover > 0:
        return 5
    else:
        return 0
    
# Covered area

def cover_score(covered_area):
    """Define the cover area threshold and score
    """
    covered_area = covered_area*100
    
    if covered_area >= 99:
        return 10
    
    elif covered_area >= 95:
        return 5
    
    else:
        return 0
    


## OPTION 1.2 Get items for all plots and store into a big df


### Selection method
The loop will search all the images between the given start-end date, and the minimum cloud coverage.<br>
After that it will calculate the sample covered area with the image item footprint and then will remove items which are under the given threshold.<br>
The next step is rank the items by the selected parameters <br>
#### Temporal selection
The user has to select the desired time span for get the images: 1 per year, 1 per month, or one every x images.

In [ ]:
# If by_month is True, one image per month will be chosen, otherwise one per year.
# By default it will process only one image per year

by_month = False
by_every = 0

### Loop over all plots in parellel
Loop over all plots and get the items.

In [ ]:
def run_multiprocess(index, row, srch_log_file, by_month=False, by_every=0, skip_items=None):
    
    aoi_geometry = json.loads(dumps(row.geometry))
    sample_id = row.name
    
    if by_every:
        pickle_df_name = os.path.join(OUT_PIKL_PATH, str(sample_id)+'_every.p')
    elif by_month:
        pickle_df_name = os.path.join(OUT_PIKL_PATH, str(sample_id)+'_month.p')
    else:
        pickle_df_name = os.path.join(OUT_PIKL_PATH, str(sample_id)+'_year.p')
        
    if not os.path.exists(pickle_df_name):
        request = build_request(aoi_geometry, start_date, stop_date, cloud_cover_lte)

        try:
            print(f'Starting {sample_id}')
            items = get_items(sample_id, request, client)
            # Transform items into a pandas dataframe with useful columns
            metadata_df = get_dataframe(items)
            
            # Skip items with errors
            if skip_items:
                metadata_df = metadata_df[~metadata_df.id.isin(skip_items)]
            
            # Mutate metadata_df and add the percentage of cover area
            add_cover_area(metadata_df, samples_gdf)

            # Remove items that are under the minimum_covered_area threshold
            metadata_df = metadata_df[metadata_df.cover_perc >= (minimum_covered_area/100)]

            # Create a score for each item
            scored_items = score_items(metadata_df, item_type_score, months_score, cloud_score, cover_score)
            
            if by_every:
                # Filter scored_items and get one item every x items
                selected_items = get_one_item_every_x(scored_items, every=by_every)
            
            elif by_month:
                # Filter scored_items and get only one per month
                selected_items = get_one_item_per_month(scored_items)
            else:
                # Filter scored_items and get only one per year
                selected_items = get_one_item_per_year(scored_items)
            
            # Save into a pickled file
            selected_items.to_pickle(pickle_df_name)
            
            print(f'{sample_id} pickled.')
            
        except Exception as e:
            print(f'there was an error with the sample {sample_id}, please check the log files.')
            with open(srch_log_file, 'a') as lf:
                lf.write(f'"{sample_id}":{e}\n')

    else:
        print(f'Search for {sample_id} already saved.')

In [ ]:
len(samples_gdf)

### Skip error items from logs
Uncomment the next cell if you have a log file with "no access to assets" elements, so the process will skip them.

<br> If you are using this option, please delete the previous searches pickled files from the failed samples (search failed samples with the commands in step 4) 

In [ ]:
skip_items = None
# skip_items = get_no_access_assets_from_log('logs/order_logs_20200916_15_02.txt')
# len(skip_items)

In [ ]:
%%time
if __name__ == '__main__':

    # Create a log file
    now = datetime.datetime.now()
    formated_now = now.strftime('%Y%m%d_%H_%M')
    srch_log_file = os.path.join(LOG_PATH, f'search_logs_{formated_now}.txt')
    
    # Set the number of parallel processes
    pool = multiprocessing.Pool(15)
    
    for index, row in samples_gdf.iterrows():
        pool.apply_async(run_multiprocess, args=(index, row, srch_log_file, by_month, by_every, skip_items))
        
    pool.close()
    pool.join()

### Read all the pickled files, merge and store them in a big df

In [ ]:
pickled_files = glob.glob(os.path.join(OUT_PIKL_PATH,'*every.p'))
len(pickled_files)

In [ ]:
all_df = pd.concat([pd.read_pickle(pkl) for pkl in pickled_files])

In [ ]:
len(all_df)

# 2. Order assets
### Create json request

In [ ]:
products_bundles = {

    # Is not possible to ask for analytic_dn in PSScene3Band, so the next option is visual
    # for more info go to https://developers.planet.com/docs/orders/product-bundles-reference/
    'PSScene3Band': "analytic,visual",
    'PSScene4Band': "analytic_udm2,analytic_sr,analytic",
    'PSOrthoTile': "analytic_5b_udm2,analytic_5b,analytic_udm2,analytic,visual",
    'REOrthoTile': "analytic,visual",
}

In [ ]:
# To create the order we need a dataframe with filtered items,
# and a samples_gdf with sample_id and geometry to clip each item.

# Build an order for each sample and store in a orders_list
orders = []
samples_ids = list(all_df.sample_id.unique())
for idx, row in samples_gdf.iterrows():
    if idx in samples_ids:
        order = build_order_from_metadata(all_df, idx, row, products_bundles)
        orders.append(order)

In [ ]:
len(orders)

### Request order
<font color='red'>The following lines will start the order in the planet server, once the order is placed and running, there is no way to stop it.</font>

NOTE: The following loop will skip the samples that have already been downloaded, however it's based on the existing_orders request, and we are not sure how long the requests will remain in the planet server.

In [ ]:
# Request the existing orders and store their sample_id (name)
current_server_orders = get_existing_orders(client)
ordered_sample_names = [order['name'] for order in current_server_orders]

now = datetime.datetime.now()
formated_now = now.strftime('%Y%m%d_%H_%M')
ordr_log_file = os.path.join(LOG_PATH, f'order_logs_{formated_now}.txt')


orders_info = []
pbar = tqdm(total=len(orders))
for new_order in orders:

    # Make sure that the sample is not already downloaded
    sample_name = new_order['name']
    if sample_name not in ordered_sample_names:
        
        try:
            # The following line will create the order in the server
            @backoff.on_exception(backoff.expo,planet.api.exceptions.OverQuota, max_time=360)
            def place_order():
                return client.create_order(new_order).get()
            
            order_info = place_order()
            orders_info.append(order_info)
            
            order_id = order_info['id']
            sample_name = order_info['name']
            
            print(f'order {order_id} with {sample_name} has been placed.')
            
        except Exception as e:
            with open(ordr_log_file, 'a') as lf:
                print(f'there was an error with the sample {sample_name}, please check the log files.')
                lf.write(f'Sample {sample_name}:{e}\n')
    pbar.update(1)
pbar.close()
print('Finished')

In [ ]:
# Use the pages to limit the search, every page will display 20 orders.
current_orders = get_orders_status(client, pages=None).sort_values(by=['created_on'])

In [ ]:
current_orders

# 3. Additional commands

In [ ]:
failed_samples = [x[0] for x in get_no_access_assets_from_log('logs/order_logs_20200915_12_41_yelena.txt')]
len(failed_samples)

In [ ]:
failed_items_ids = [x[1] for x in get_no_access_assets_from_log('logs/order_logs_20200915_12_41_yelena.txt')]
failed_items_ids

In [5]:
from IPython.display import Audio
sound_file = 'http://soundbible.com/mp3/Home%20Phone%20Ringing-SoundBible.com-476855293.mp3'
Audio(sound_file, autoplay=True)

In [7]:
!curl -L -H "Authorization: api-key df2cea02b871467084d091b61361ce1c" \
    'https://api.planet.com/auth/v1/experimental/public/my/subscriptions'

[
  {
    "active_from": "2019-09-23T00:00:00+00:00", 
    "active_to": "2020-09-22T00:00:00+00:00", 
    "basemap_quad_quota": null, 
    "basemap_tile_quota": null, 
    "created_at": "2019-12-24T15:34:16.364433+00:00", 
    "datadrop_anchor_date": "2020-02-01T00:00:00+00:00", 
    "datadrop_enabled": false, 
    "datadrop_interval": null, 
    "deleted_at": null, 
    "id": 360334, 
    "organization": {
      "id": 203769, 
      "name": "Indonesia_FAO_Countries"
    }, 
    "organization_id": 203769, 
    "plan": {
      "id": 6025, 
      "name": "PL-015973_Indonesia", 
      "state": "active"
    }, 
    "plan_id": 6025, 
    "quota_anchor_date": "2020-02-01T00:00:00+00:00", 
    "quota_enabled": true, 
    "quota_interval": null, 
    "quota_reset_at": null, 
    "quota_sqkm": 171984, 
    "quota_style": "consumption", 
    "quota_used": 157998.863192325, 
    "reference": "PL-015973", 
    "scene_tile_quota": null, 
    "selected_operations": null, 
    "state": "active", 
   

In [ ]:
Audio(sound_file, autoplay=True)